In [6]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tqdm.contrib import itertools
from sklearn.preprocessing import LabelEncoder
import math


In [7]:
train_dir = './small_dog_cat_dataset/train/'
test_dir = './small_dog_cat_dataset/test/'

In [33]:
train_data = []
train_labels = []
test_data = []
test_labels = []

target_size = (64,64)

In [9]:
def read_file(path,datas,labels):
    for category in os.listdir(path):
     category_dir = os.path.join(path, category)
     for image_name in os.listdir(category_dir):
        image_path = os.path.join(category_dir, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, target_size)
        datas.append(image)
        labels.append(category)
    return np.array(datas), np.array(labels)

In [34]:
read_file(train_dir,train_data,train_labels)
read_file(test_dir,test_data,test_labels)

(array([[[[ 64,  64,  64],
          [ 57,  57,  57],
          [ 60,  60,  60],
          ...,
          [190, 190, 190],
          [192, 192, 192],
          [191, 191, 191]],
 
         [[ 68,  68,  68],
          [ 56,  56,  56],
          [ 61,  61,  61],
          ...,
          [191, 191, 191],
          [193, 193, 193],
          [193, 193, 193]],
 
         [[ 62,  62,  62],
          [ 58,  58,  58],
          [ 65,  65,  65],
          ...,
          [192, 192, 192],
          [194, 194, 194],
          [194, 194, 194]],
 
         ...,
 
         [[157, 162, 165],
          [154, 158, 161],
          [148, 151, 154],
          ...,
          [171, 174, 179],
          [170, 173, 178],
          [170, 173, 178]],
 
         [[167, 169, 170],
          [195, 197, 198],
          [187, 189, 190],
          ...,
          [169, 173, 178],
          [168, 171, 176],
          [166, 169, 175]],
 
         [[196, 198, 199],
          [196, 198, 199],
          [200, 202, 203],
   

In [11]:
def colorspace(data):
   hsv_images = []
   for image in data:
        hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        hsv_images.append(hsv)
   return np.array(hsv_images)

In [12]:
train_hsv_images = colorspace(train_data)
test_hsv_images = colorspace(test_data)


In [35]:
train_labels = LabelEncoder().fit_transform(train_labels)

In [36]:
test_labels = LabelEncoder().fit_transform(test_labels)

In [16]:
def HOGhsvfearture(data):
    hog_hsv_features =[]
    for image in data:
        # hue = image[:,:,0]
        hog_features, hog_image = hog(image, visualize=True,
                                       block_norm='L2-Hys',
                                         pixels_per_cell=(16, 16),
                                           cells_per_block=(2, 2),
                                           channel_axis=-1)
        hog_hsv_features.append(hog_features)
    return hog_hsv_features

In [17]:
train_hsv_feature = HOGhsvfearture(train_hsv_images)
test_hsv_feature = HOGhsvfearture(test_hsv_images)


In [18]:
train_hsv_feature = np.array(train_hsv_feature)
test_hsv_feature = np.array(test_hsv_feature)

In [19]:
def logistic(x):
    """
    Computes the logistic function applied to an input scalar/array
    Args:
        x (scalar/ndarray): scalar or numpy array of any size
    Returns:
        y (scalar/ndarray): logistic function applied to x, has the same shape as x
    """
    y = 1 / (1 + np.exp(-x))
    return y

In [20]:
def log_loss(y, y_dash):
    """
    Computes log loss for inputs true value (0 or 1) and predicted value (between 0 and 1)
    Args:
      y      (scalar): true value (0 or 1)
      y_dash (scalar): predicted value (probability of y being 1)
    Returns:
      loss (float): nonnegative loss corresponding to y and y_dash
    """
    loss = - (y * np.log(y_dash)) - ((1 - y) * np.log(1 - y_dash))
    return loss

In [21]:
def cost_func_vec(y, y_dash):
    """
    Computes log loss for inputs true value (0 or 1) and predicted value (between 0 and 1)
    Args:
      y      (array_like, shape (m,)): array of true values (0 or 1)
      y_dash (array_like, shape (m,)): array of predicted values (probability of y being 1)
    Returns:
      cost (float): nonnegative cost corresponding to y and y_dash
    """
    assert len(y) == len(y_dash), "Length of true values and length of predicted values do not match"
    m = len(y)
    loss_vec = np.array([log_loss(y[i], y_dash[i]) for i in range(m)])
    cost = np.dot(loss_vec, np.ones(m)) / m
    return cost

In [22]:
def cost_logreg_vec(X, y, w, b):
    """
    Computes the cost function, given data and model parameters
    Args:
      X (ndarray, shape (m,n))  : data on features, m observations with n features
      y (array_like, shape (m,)): array of true values of target (0 or 1)
      w (array_like, shape (n,)): weight parameters of the model      
      b (float)                 : bias parameter of the model
    Returns:
      cost (float): nonnegative cost corresponding to y and y_dash 
    """
    m, n = X.shape
    assert len(y) == m, "Number of feature observations and number of target observations do not match"
    assert len(w) == n, "Number of features and number of weight parameters do not match"
    z = np.matmul(X, w) + (b * np.ones(m))
    y_dash = logistic(z)
    cost = cost_func_vec(y, y_dash)
    return cost

In [23]:
def grad_logreg_vec(X, y, w, b): 
    """
    Computes gradients of the cost function with respect to model parameters
    Args:
      X (ndarray, shape (m,n))  : data on features, m observations with n features
      y (array_like, shape (m,)): array of true values of target (0 or 1)
      w (array_like, shape (n,)): weight parameters of the model      
      b (float)                 : bias parameter of the model
    Returns:
      grad_w (array_like, shape (n,)): gradients of the cost function with respect to the weight parameters
      grad_b (float)                 : gradient of the cost function with respect to the bias parameter
    """
    m, n = X.shape
    assert len(y) == m, "Number of feature observations and number of target observations do not match"
    assert len(w) == n, "Number of features and number of weight parameters do not match"
    y_dash = logistic(np.matmul(X, w) + b * np.ones(m))
    grad_w = np.matmul(y_dash - y, X) / m
    grad_b = np.dot(y_dash - y, np.ones(m)) / m
    return grad_w, grad_b

In [24]:
def cost_logreg_vec_reg(X, y, w, b, l):
    """
    Computes the cost function, given data and model parameters
    Args:
      X (ndarray, shape (m,n))  : data on features, m observations with n features
      y (array_like, shape (m,)): array of true values (0 or 1) of target
      w (array_like, shape (n,)): weight parameters of the model      
      b (float)                 : bias parameter of the model
      l (float)                 : regularization parameter
    Returns:
      cost (float): nonnegative cost corresponding to y and y_dash 
    """
    m, n = X.shape
    assert len(y) == m, "Number of feature observations and number of target observations do not match"
    assert len(w) == n, "Number of features and number of weight parameters do not match"
    cost = cost_logreg_vec(X, y, w, b)
    cost += (l / (2 * m)) * np.dot(w, w)
    return cost

In [25]:
def grad_logreg_vec_reg(X, y, w, b, l):
    """
    Computes gradients of the cost function with respect to model parameters
    Args:
      X (ndarray, shape (m,n))  : data on features, m observations with n features
      y (array_like, shape (m,)): array of true values of target (0 or 1)
      w (array_like, shape (n,)): weight parameters of the model      
      b (float)                 : bias parameter of the model
      l (float)                 : regularization parameter
    Returns:
      grad_w (array_like, shape (n,)): gradients of the cost function with respect to the weight parameters
      grad_b (float)                 : gradient of the cost function with respect to the bias parameter
    """
    m, n = X.shape
    assert len(y) == m, "Number of feature observations and number of target observations do not match"
    assert len(w) == n, "Number of features and number of weight parameters do not match"
    grad_w, grad_b = grad_logreg_vec(X, y, w, b)
    grad_w += (l / m) * w
    return grad_w, grad_b

In [26]:
def create_mini_batches(X, y, batch_size):
  mini_batches = []
  n_minibatches = X.shape[0]
  i = 0
  for i in range(n_minibatches + 1):
    X_mini = X[i * batch_size:(i + 1)*batch_size, :]
    Y_mini = y[i * batch_size:(i + 1)*batch_size]
    if X_mini.shape[0] == 0:
      break
    mini_batches.append((X_mini, Y_mini))
  return mini_batches

In [40]:
X_hsv_train, X_hsv_val, y_hsv_train, y_hsv_val, = train_test_split(train_hsv_feature,
                                                                    train_labels,
                                                                      test_size=0.1,
                                                                        random_state=50)

In [28]:
def grad_desc_reg_with_mini_batch(X, y, w, b, l, alpha, n_iter, show_cost = True, show_params = False, batch_size=32): 
    """
    Implements batch gradient descent algorithm to learn and update model parameters
    with prespecified number of interations and learning rate
    Args:
      X (ndarray, shape (m,n))  : data on features, m observations with n features
      y (array_like, shape (m,)): true values of target (0 or 1)
      w (array_like, shape (n,)): initial value of weight parameters
      b (scalar)                : initial value of bias parameter
      l (float)                 : regularization parameter
      alpha (float)             : learning rate
      n_iter (int)              : number of iterations
    Returns:
      w (array_like, shape (n,)): updated values of weight parameters
      b (scalar)                : updated value of bias parameter
    """
    m, n = X.shape
    assert len(y) == m, "Number of feature observations and number of target observations do not match"
    assert len(w) == n, "Number of features and number of weight parameters do not match"
    cost_history, params_history = [], []
    for i, j in itertools.product(range(n_iter), range(1)):
        mini_batches = create_mini_batches(X,y,batch_size)
        for mini_batch in mini_batches:
          X_mini, y_mini = mini_batch
          grad_w, grad_b = grad_logreg_vec_reg(X_mini, y_mini, w, b, l)   
          w += - alpha * grad_w
          b += - alpha * grad_b
          cost =  cost_logreg_vec_reg(X_mini, y_mini, w, b, l)
          cost_history.append(cost)
          params_history.append([w, b])

        if show_cost == True and show_params == False and (i % math.ceil(n_iter / 10) == 0 or i == n_iter - 1):
            print(f"Iteration {i:6}:    Cost  {float(cost_history[i]):3.4f}")
        if show_cost == True and show_params == True and (i % math.ceil(n_iter / 10) == 0 or i == n_iter - 1):
            print(f"Iteration {i:6}:    Cost  {float(cost_history[i]):3.4f},    Params  {params_history[i]}")
    return w, b, cost_history, params_history

In [323]:
def grad_desc_reg(X, y, w, b, l, alpha, n_iter, show_cost = True, show_params = False): 
    """
    Implements batch gradient descent algorithm to learn and update model parameters
    with prespecified number of interations and learning rate
    Args:
      X (ndarray, shape (m,n))  : data on features, m observations with n features
      y (array_like, shape (m,)): true values of target (0 or 1)
      w (array_like, shape (n,)): initial value of weight parameters
      b (scalar)                : initial value of bias parameter
      l (float)                 : regularization parameter
      alpha (float)             : learning rate
      n_iter (int)              : number of iterations
    Returns:
      w (array_like, shape (n,)): updated values of weight parameters
      b (scalar)                : updated value of bias parameter
    """
    m, n = X.shape
    assert len(y) == m, "Number of feature observations and number of target observations do not match"
    assert len(w) == n, "Number of features and number of weight parameters do not match"
    cost_history, params_history = [], []
    for i, j in itertools.product(range(n_iter), range(1)):
        grad_w, grad_b = grad_logreg_vec_reg(X, y, w, b, l)   
        w += - alpha * grad_w
        b += - alpha * grad_b
        cost =  cost_logreg_vec_reg(X, y, w, b, l)
        cost_history.append(cost)
        params_history.append([w, b])
        if show_cost == True and show_params == False and (i % math.ceil(n_iter / 10) == 0 or i == n_iter - 1):
            print(f"Iteration {i:6}:    Cost  {float(cost_history[i]):3.4f}")
        if show_cost == True and show_params == True and (i % math.ceil(n_iter / 10) == 0 or i == n_iter - 1):
            print(f"Iteration {i:6}:    Cost  {float(cost_history[i]):3.4f},    Params  {params_history[i]}")
    return w, b, cost_history, params_history

In [29]:
# Initial values of the model parameters
# Initial values of the model parameters
def create_weight():
    w_init = np.zeros(train_hsv_feature.shape[1]).astype(float)
    b_init = -1.
    return w_init, b_init

In [30]:
w_init, b_init = create_weight()

In [41]:
# Learning model parameters using gradient descent algorithm
w_out_reg, b_out_reg, cost_history_reg, params_history_reg = grad_desc_reg_with_mini_batch(X_hsv_train,
                                                                           y_hsv_train,
                                                                           w = w_init, # np.zeros(X_train.shape[1]),
                                                                           b = b_init, # 0,
                                                                           l = 1.,
                                                                           alpha = 0.1,
                                                                           n_iter = 4000)

  0%|          | 2/4000 [00:00<05:43, 11.63it/s]

Iteration      0:    Cost  0.7258


 10%|█         | 404/4000 [00:30<04:10, 14.34it/s]

Iteration    400:    Cost  0.6815


 20%|██        | 804/4000 [00:58<03:28, 15.35it/s]

...

Iteration   3200:    Cost  0.6815


 90%|█████████ | 3604/4000 [04:08<00:24, 16.01it/s]

Iteration   3600:    Cost  0.6611


100%|██████████| 4000/4000 [04:32<00:00, 14.69it/s]

Iteration   3999:    Cost  0.6611


In [343]:
y_train_prob_reg = logistic(np.matmul(X_hsv_val, w_out_reg) + (b_out_reg * np.ones(X_hsv_val.shape[0])))

In [344]:
y_test_prob_reg = logistic(np.matmul(test_hsv_feature, w_out_reg) + (b_out_reg * np.ones(test_hsv_feature.shape[0])))

In [345]:
y_test_prob_reg =  (y_test_prob_reg > 0.5).astype(int)

In [42]:
def hypothesis(X, w, b):
    y_prob = logistic(np.matmul(X, w) + (b * np.ones(X.shape[0])))
    y_pred = (y_prob > 0.5).astype(int)
    return y_pred

In [346]:
y_train_pred_reg =  (y_train_prob_reg > 0.5).astype(int)

In [43]:
y_val_pred = hypothesis(X_hsv_val, w_out_reg, b_out_reg)

In [44]:
from sklearn.metrics import accuracy_score

acc_pred = accuracy_score(y_hsv_val, y_val_pred)
print("Accuracy test:", acc_pred)

Accuracy test: 0.71


In [45]:
y_test_pred = hypothesis(test_hsv_feature, w_out_reg, b_out_reg)

In [46]:
acc_pred = accuracy_score(test_labels, y_test_pred)
print("Accuracy test:", acc_pred)

Accuracy test: 0.6866666666666666
